# Explore the COAWST US East Coast and Gulf of Mexico Forecast Archive Dataset
This is a cloud-optimized version of the NetCDF files accessed from the USGS ScienceBase item [Collection of COAWST model forecast for the US East Coast and Gulf of Mexico](https://www.sciencebase.gov/catalog/item/610acd4fd34ef8d7056893da).   The original daily forecast files were converted into weekly NetCDF files with 168 points in the time dimension to facilitate time series access. 

In [ ]:
import os
import fsspec
import xarray as xr
import hvplot.xarray
import intake
import cf_xarray
import numpy as np
import panel as pn
from matplotlib import path
import xoak

## Open Dataset

The details of data loading are stored in an `intake` catalog, which simplifies use.  Metadata and coordinate data are loaded, but not the actual data variables, which are loaded only as needed by subsequent analysis and visualization. 

In [ ]:
intake_catalog_url = 's3://usgs-coawst/coawst_us_east_and_gulf.yml'
cat = intake.open_catalog(intake_catalog_url, storage_options={"anon": True} )
list(cat)

In [ ]:
dataset = 'COAWST_USEAST_Archive' 

This is a big dataset, so it takes up to 30s to open the dataset (which involves reading all the metadata and index coordinate variable data). Here we load the data into xarray using `.to_dask()` so that if we have a Dask cluster, we can speed up data processing by loading and processing chunks of data in parallel. 

In [ ]:
%%time
ds = cat[dataset].to_dask()

Let's look at that metadata.  We can explore the different attributes and variables by clicking on the variables and icons below. 

In [ ]:
ds

We can also explore a specific variable of interest:

In [ ]:
var = 'Hwave'
da = ds[var]
da

Use the CF conventions to identify the coordinate variables for longitude, latitude and time

In [ ]:
da = ds['Hwave']

In [ ]:
x = da.cf['longitude']
y = da.cf['latitude']
t = da.cf['time']
print(x.name, y.name, t.name)

In [ ]:
da.sel(ocean_time='2012-10-29 12:00', method='nearest')

## Example A: Load the entire spatial domain for a variable at a specific time step
Loading the entire spatial domain 


In [ ]:
%%time
da2d = da.sel(ocean_time='2012-10-29 12:00', method='nearest').load()

In [ ]:
da2d.hvplot.quadmesh(x=x.name, y=y.name, rasterize=True, geo=True, tiles='OSM', cmap='viridis')

## Example B: Load a time series for a variable at a specific lon,lat location for a specified time range. 
We can see how many chunks of data must be loaded to extract a time series inspecting the dataset that will result:

### Parallelize with Dask 
We opened the dataset so that we can take advantage of parallel compute environments
using `dask`. We're going to start a cluster now so that future steps can take advantage
of this ability. 

This is an optional step, but speeds up data loading and processing significantly, especially 
when accessing data from the cloud.

In [ ]:
%run Start_Dask_Cluster_Nebari.ipynb

In [ ]:
cluster.adapt(minimum=2, maximum=10)

In [ ]:
client

To identify a point, we will start with its lat/lon coordinates.  If lon and lat were 1D coordinates, we could use lon,lat values to select using xarray, but instead we need to extract using indices, which we need to find.   For this we use the `xoak` package:

In [ ]:
lat,lon = 42.0, -69.0

In [ ]:
da.xoak.set_index([y.name, x.name], 'scipy_kdtree')

In [ ]:
ds_point = xr.Dataset({"lon": ("point", [lon]), "lat": ("point", [lat])})

Load one month:

In [ ]:
%%time
ds_selection = da.xoak.sel(lat_rho=ds_point.lat, lon_rho=ds_point.lon).sel(ocean_time='2012-10').load()

In [ ]:
ds_selection.hvplot(x=t.name, grid=True)

Load the entire time series:

In [ ]:
%%time
ds_selection = da.xoak.sel(lat_rho=ds_point.lat, lon_rho=ds_point.lon).load()
ds_selection.hvplot(x=t.name)                       

## Example C: Compute the time mean for a variable over the entire domain for a specific time period

In [ ]:
%%time
da_mean = da.sel(ocean_time=slice('2016-01-01 00:00','2017-01-01 00:00')).mean(dim='ocean_time').compute()

In [ ]:
da_mean.hvplot.quadmesh(x=x.name, y=y.name, rasterize=True, geo=True,  tiles='OSM', frame_width=600, cmap='viridis')

## Example D: Subset a time and space region and export to NetCDF

In [ ]:
def bbox2ij(lon,lat,bbox=[-160., -155., 18., 23.]):
    """Return indices for i,j that will completely cover the specified bounding box.     
    i0,i1,j0,j1 = bbox2ij(lon,lat,bbox)
    lon,lat = 2D arrays that are the target of the subset
    bbox = list containing the bounding box: [lon_min, lon_max, lat_min, lat_max]

    Example
    -------  
    >>> i0,i1,j0,j1 = bbox2ij(lon_rho,lat_rho,[-71, -63., 39., 46])
    >>> h_subset = nc.variables['h'][j0:j1,i0:i1]       
    """
    bbox=np.array(bbox)
    mypath=np.array([bbox[[0,1,1,0]],bbox[[2,2,3,3]]]).T
    p = path.Path(mypath)
    points = np.vstack((lon.ravel(),lat.ravel())).T   
    n,m = np.shape(lon)
    inside = p.contains_points(points).reshape((n,m))
    ii,jj = np.meshgrid(range(m),range(n))
    return min(ii[inside]),max(ii[inside]),min(jj[inside]),max(jj[inside])

In [ ]:
bbox = [-76.63290610753754, -73.55671530588432, 37.57888442021855, 41.225532965406224]   # DRB

In [ ]:
i0,i1,j0,j1 = bbox2ij(x.values, y.values, bbox=bbox)
print(i0,i1,j0,j1)

In [ ]:
ds_drb = ds[['temp', 'salt', 'Hwave']].isel(eta_rho=slice(j0,j1), xi_rho=slice(i0,i1))

In [ ]:
ds_drb

In [ ]:
ds_drb_timeslice = ds_drb.sel(ocean_time=slice('2022-04-01 00:00','2022-04-08 00:00'))

In [ ]:
ds_drb_timeslice = ds_drb_timeslice.chunk({'eta_rho':-1, 'xi_rho':-1})  # chunk to full spatial subset domain
print(f'Uncompressed dataset size: {ds_drb_timeslice.nbytes/1e6} MB')

In [ ]:
var = 'Hwave'
da_drb = ds_drb_timeslice[var]

In [ ]:
viz = da_drb.hvplot.quadmesh(x=x.name, y=y.name, geo=True, frame_width=600,
                    cmap='turbo', rasterize=True, tiles='OSM', title=var)
viz = pn.panel(viz, widgets={'ocean_time': pn.widgets.Select} )
pn.Column(viz).servable('DRB Explorer')

In [ ]:
fs = fsspec.filesystem('file', skip_instance_cache=True)

In [ ]:
%%time
encoding={}
for var in ds_drb_timeslice.variables:
    encoding[var] = dict(zlib=True, complevel=4, 
                         fletcher32=False, shuffle=True,
                         _FillValue=None
                        )
ds_drb_timeslice.to_netcdf('drb.nc', encoding=encoding, mode='w', engine='h5netcdf')

fsize = fs.size('drb.nc')/1e6
print(f'NetCDF file size: {fsize} MB')

## Stop cluster

In [ ]:
client.close(); cluster.shutdown()